# Senior Cubers Worldwide - Weekly Competition

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/wca-ipy/

## Initialisation

Basic approach to determine the project directory

In [1]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

## Formatting Functions

Functions to convert results to and from seconds or display an age category

In [2]:
def numSeconds(value):
    '''Convert float or string to number of seconds - e.g. 1:05.31 returns 65.31'''   
    
    if isinstance(value, float):
        return round(value, 2)
    elif ':' in value:
        parts = value.split(':')
        return round(int(parts[0]) * 60 + float(parts[1]), 2)
    elif 'n/a' in value.lower():
        return 0
    elif 'dnf' in value.lower():
        return -1
    elif 'dns' in value.lower():
        return -2
    else:
        return None


def formatResult(value, eventName, highlight = ''):
    '''Convert number of seconds to displayable time - e.g. 65.31 returns 1:05.31'''
    
    if value is not None:
        if value > 0:
            if eventName.lower() == 'fmc':
                return '{}{:d}{}'.format(highlight, int(value), highlight)
            else:
                if value > 60:
                    return '{}{:d}:{:05.2f}{}'.format(highlight, int(value // 60), value - int(value // 60) * 60, highlight)
                else:
                    return '{}{:.2f}{}'.format(highlight, value, highlight)
        else:
            if value == 0:
                return '-'
            elif value == -1:
                return 'DNF'
            elif value == -2:
                return 'DNS'
            else:
                return '?'
    else:
        return ''

In [3]:
def formatAge(age):
    '''Format age for report'''
    
    if age < 40:
        return '<{}'.format(age + 10)
    else:
        return '{}+'.format(age)

    
def formatAgeLong(age):
    '''Format age for report'''
    
    if age < 40:
        return 'Under {}'.format(age + 10)
    else:
        return 'Over {}'.format(age)

In [4]:
def getSafeName(name):
    '''Return name which is safe for URL'''
    
    nameDecomposed = unicodedata.normalize('NFKD', name)
    safeName = nameDecomposed.encode('ascii', 'ignore').decode('ascii').strip()
    
    if len(safeName) == 0:
        safeName = name

    return safeName.lower().replace(' ', '_')

In [5]:
def writeGoogleSiteTag(f):
    '''Write Google Site Tag'''

    comment = '<!-- Global site tag (gtag.js) - Google Analytics -->'
    src = '<script async src="https://www.googletagmanager.com/gtag/js?id=UA-86348435-3"></script>'
    gtag = "<script>window.dataLayer = window.dataLayer || []; function gtag() {dataLayer.push(arguments);} gtag('js', new Date()); gtag('config', 'UA-86348435-3');</script>"

    f.write('\n{}\n{}\n{}\n'.format(comment, src, gtag))

In [6]:
title = 'Senior Cubers Worldwide - Weekly Competition'

In [7]:
eventNames = \
[
    '3x3x3',
    '2x2x2',
    '4x4x4',
    '5x5x5',
    'OH',
    '3BLD',
    'FMC'
]

## Generic Class

Generic class to ensure that all custom classes are printable

In [8]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Result Class

Simple class for a competition result

In [9]:
class Result(Printable):
    def __init__(self, date, rank, name, age, result, single, average, awards, solves, link):
        self.date, self.rank = date, rank
        self.name, self.age = name, age
        self.result, self.single, self.average = result, single, average
        self.awards = awards
        self.solves, self.link = solves, link
        
        
    def saveHeader(self, f, eventName):
        '''Save header for table'''
        
        f.write('| # | Name | Age | Single |')
        if eventName.lower() != 'fmc':
            f.write(' Average |')
        f.write(' Awards |')
        for i in range(len(self.solves)):
            f.write(' Solve {} |'.format(i + 1))
        if eventName.lower() == 'fmc':
            f.write(' Solution |\n')
        else:
            f.write(' Video |\n')

        f.write('| :--: | -- | :--: |')
        if eventName.lower() != 'fmc':
            f.write(' --: | --: |')
        else:
            f.write(' :--: |')
        f.write(' :--: |')
        for i in self.solves:
            f.write(' --: |')
        f.write(' :-- |\n')

        
    def saveResult(self, f, eventName):
        '''Save header for table'''
        
        link = '[{}](../persons/{}.md)'.format(self.name, getSafeName(self.name))
        f.write('| {:d} | {} | {} |'.format(self.rank, link, formatAge(self.age)))
        f.write(' {} |'.format(formatResult(self.single, eventName)))
        if eventName.lower() != 'fmc':
            f.write(' {} |'.format(formatResult(self.average, eventName)))
        f.write(' {} |'.format(self.awards))
        for solve in self.solves:
            f.write(' {} |'.format(formatResult(solve, eventName)))
        if 'http' in self.link:
            f.write(' [Link]({}) |'.format(self.link))
        else:
            f.write(' |')
        f.write('\n')

## Event Class

Simple class to hold a list of results

In [10]:
class Event(Printable):
    def __init__(self, name):
        self.name = name
        self.results = []
        self.maxSolves = 0

        if name.lower() == 'oh':
            self.longName = '3x3x3 One-Handed'
            self.shortName = '3x3x3 OH'
        elif name.lower() == '3bld':
            self.longName = '3x3x3 Blindfolded'
            self.shortName = '3x3x3 BLD'
        elif name.lower() == 'fmc':
            self.longName = '3x3x3 Fewest Moves'
            self.shortName = '3x3x3 FMC'
        else:
            self.longName = name[0].upper() + name[1:]
            self.shortName = self.longName
        
    def addResult(self, result):
        self.results.append(result)

## Person Class

Results for a single person 

In [11]:
import unicodedata

class Person(Printable):
    def __init__(self, name):
        self.name = name
        self.age = 0
        self.bestSingles = {}
        self.bestAverages = {}
        self.events = {}


    def isBestSingle(self, eventName, result):
        if result.single:
            if eventName in self.bestSingles:
                if self.bestSingles[eventName].single > 0:
                    if result.single > 0 and self.bestSingles[eventName].single >= result.single:
                        self.bestSingles[eventName] = result
                        return True
                else:
                    if result.single > 0 and self.bestSingles[eventName].single <= result.single:
                        self.bestSingles[eventName] = result
                        return True
            else:
                self.bestSingles[eventName] = result

        return False
    
    
    def isBestAverage(self, eventName, result):
        if result.average:
            if eventName in self.bestAverages:
                if self.bestAverages[eventName].average > 0:
                    if result.average > 0 and self.bestAverages[eventName].average >= result.average:
                        self.bestAverages[eventName] = result
                        return True
                else:
                    if result.average > 0 and self.bestAverages[eventName].average <= result.average:
                        self.bestAverages[eventName] = result
                        return True
            else:
                self.bestAverages[eventName] = result

        return False

    
    def addResult(self, eventName, result):
        if eventName in self.events:
            event = self.events[eventName]
        else:
            event = Event(eventName)
            self.events[eventName] = event

        event.addResult(result)

        if result.age > self.age:
            self.age = result.age
            
            
    def saveEventHeader(self, f, event):
        '''Save header for table'''
        
        f.write('\n#### {}\n\n'.format(event.longName))

        f.write('| Date | Age | Single |')
        if event.name.lower() != 'fmc':
            f.write(' Average |')
        f.write(' Awards |')
        for i in range(event.maxSolves):
            f.write(' Solve {} |'.format(i + 1))
        if event.name.lower() == 'fmc':
            f.write(' Solution |\n')
        else:
            f.write(' Video |\n')

        f.write('| :--: | :--: |')
        if event.name.lower() != 'fmc':
            f.write(' --: | --: |')
        else:
            f.write(' :--: |')
        f.write(' :--: |')
        for i in range(event.maxSolves):
            f.write(' --: |')
        f.write(' :-- |\n')

        
    def saveEventResult(self, f, event, result, count):
        '''Save header for table'''
        
        link = '[{}](../{}/{}.md)'.format(result.date, getSafeName(event.name), result.date)
        f.write('| {} | {} |'.format(link, formatAge(result.age)))
        if '⚡' in result.awards or count == 1:
            highlight = '**'
        else:
            highlight = ''
        f.write(' {} |'.format(formatResult(result.single, event.name, highlight)))
        if event.name.lower() != 'fmc':
            if '🔥' in result.awards or count == 1:
                highlight = '**'
            else:
                highlight = ''
            f.write(' {} |'.format(formatResult(result.average, event.name, highlight)))
        f.write(' {} |'.format(result.awards))
        for i in range(event.maxSolves):
            if len(result.solves) > i:
                if result.solves[i] > 0 and result.solves[i] == result.single and ('⚡' in result.awards or count == 1):
                    highlight = '**'
                else:
                    highlight = ''
                f.write(' {} |'.format(formatResult(result.solves[i], event.name, highlight)))
            else:
                f.write(' - |')
        if 'http' in result.link:
            f.write(' [Link]({}) |'.format(result.link))
        else:
            f.write(' |')
        f.write('\n')

        
    def saveProfile(self):
        '''Save profile as markdown'''

        docsDir = os.path.join(projdir, 'docs', 'persons')
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, getSafeName(self.name) + '.md')

        with open(outFile, 'w') as f:
            
            f.write('## {}\n'.format(title))
            f.write('### {}\n\n'.format(self.name))

            f.write('| Event | Single | Average |\n')
            f.write('| -- | --: | --: |\n')
            
            for eventName in eventNames:
                if eventName in self.events:
                    event = Event(eventName)
                    f.write('| {} |'.format(event.shortName))
                    if eventName in self.bestSingles:
                        f.write(' {} |'.format(formatResult(self.bestSingles[eventName].single, eventName)))
                    else:
                        f.write(' - |')
                    if eventName in self.bestAverages:
                        f.write(' {} |'.format(formatResult(self.bestAverages[eventName].average, eventName)))
                    else:
                        f.write(' - |')
                    f.write('\n')

            for eventName in eventNames:
                if eventName in self.events:
                    event = self.events[eventName]
                    
                    event.maxSolves = 0
                    for result in event.results:
                        if len(result.solves) > event.maxSolves:
                            event.maxSolves = len(result.solves)

                    self.saveEventHeader(f, event)
                    count = len(event.results)
                    for result in reversed(event.results):
                        self.saveEventResult(f, event, result, count)
                        count -= 1

                    f.write('\n')

            writeGoogleSiteTag(f)

## Competition Class

Results for a single competition 

In [12]:
from xlrd import open_workbook
from datetime import datetime

import Levenshtein

class Competition(Printable):
    def __init__(self, xslx):
        self.xlsx = xslx
        self.date = os.path.basename(os.path.dirname(xslx))
        self.events = {}

        
    def saveEvent(self, eventName):
        '''Save results as markdown'''

        event = self.events[eventName]
        results = event.results
        
        docsDir = os.path.join(projdir, 'docs', getSafeName(eventName))
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, self.date + '.md')

        with open(outFile, 'w') as f:
            
            f.write('## {}\n'.format(title))
            f.write('### {} {}\n\n'.format(event.longName, self.date))
            
            results[0].saveHeader(f, eventName)
            for result in results:
                result.saveResult(f, eventName)

            writeGoogleSiteTag(f)


    def saveEvents(self):
        '''Save results as markdown'''

        for eventName in self.events:
            self.saveEvent(eventName)


    def processSheets(self):
        '''Process a spreadsheet which has been downloaded from Google Sheets'''

        wb = open_workbook(self.xlsx)

        for sheet in wb.sheets():
            # Fixed columns
            indexRank = 0
            indexName = 1
            indexAge = 2
            indexResult = 3

            # Variable columns
            resultName = None
            labelAverage = None
            indexAverage = None
            indexAwards = None
            indexSolves = []
            indexLink = None
            
            # Counters, etc
            rowNo = 1
            prevResult = prevSingle = 0
            results = []

            eventName = sheet.cell(0, 0).value
            event = Event(eventName)
            self.events[eventName] = event
            
            for row in range(sheet.nrows):
                values = []
                for col in range(sheet.ncols):
                    values.append(sheet.cell(row, col).value)

                # Process header row - must be row 3
                if rowNo == 3:
                    for i in range(len(values)):
                        if str(values[i]).startswith('Award'):
                            indexAwards = i
                        elif str(values[i]).startswith('Solve'):
                            indexSolves.append(i)
                        elif 'Mo3' in str(values[i]):
                            labelAverage = values[i]
                            indexAverage = i
                        elif 'Ao5' in str(values[i]):
                            labelAverage = values[i]
                            indexAverage = i
                        elif 'link' in str(values[i]).lower():
                            indexLink = i
                    resultName = values[indexResult]

                # Process result row - must be after header on row 3
                elif rowNo > 3:
                    # Fixed fields
                    name = values[indexName].strip()
                    age = values[indexAge].lower()
                    if 'under' in age:
                        age = int(age.replace('under', '').replace(' ', '')) - 10
                    else:
                        age = int(age.replace('+', ''))
                    
                    # Process solves - determine single, average, etc.
                    solves = []
                    single = None
                    for indexSolve in indexSolves:
                        value = numSeconds(values[indexSolve])
                        if value and value > 0:
                            if single is None or single > value:
                                single = value
                        solves.append(value)
                    if single == None:
                        single = numSeconds(values[indexResult])
                    if indexAverage:
                        average = numSeconds(values[indexAverage])
                    else:
                        average = None
                    awards = values[indexAwards].strip()
                    link = values[indexLink].strip()
                    
                    # Determine result in seconds and rank
                    thisResult = numSeconds(values[indexResult])
                    if thisResult != prevResult or single != prevSingle:
                        rank = rowNo - 3

                    # Check person
                    if name in persons:
                        person = persons[name]
                    else:
                        for person in persons:
                            if (Levenshtein.distance(person, name) < 5):
                                print("WARNING: Similar names -", person, "+", name)
                        person = Person(name)
                        persons[name] = person

                    # Record result
                    result = Result(self.date, rank, name, age, thisResult, single, average, awards, solves, link)
                    results.append(result)
                    person.addResult(eventName, result)
                    event.addResult(result)

                    # Check rank
                    if (thisResult > 0 and thisResult < prevResult):
                        print('ERROR: Order is incorrect for {} in {} {}'.format(name, eventName, self.date))
                    if (values[indexRank] != rank):
                        print('ERROR: Rank is incorrect for {} in {} {}'.format(name, eventName, self.date))

                    # Check single
                    if person.isBestSingle(eventName, result):
                        if '⚡' not in awards:
                            print('WARNING: Missing ⚡ for {} in {} {}'.format(name, eventName, self.date))
                    else:
                        if '⚡' in awards:
                            print('WARNING: Spurious ⚡ for {} in {} {}'.format(name, eventName, self.date))

                    # Check average
                    if person.isBestAverage(eventName, result):
                        if '🔥' not in awards:
                            print('WARNING: Missing 🔥 for {} in {} {}'.format(name, eventName, self.date))
                    else:
                        if '🔥' in awards:
                            print('WARNING: Spurious 🔥 for {} in {} {}'.format(name, eventName, self.date))

                    # Copy list of solves prior to manipulation
                    tmpSolves = solves.copy()
                    
                    # Remove DNS - e.g. doing Mo3 rather than Ao5
                    while -2 in tmpSolves:
                        tmpSolves.remove(-2)
                        
                    # Remove n/a - e.g. doing Mo3 rather than Ao5
                    while 0 in tmpSolves:
                        tmpSolves.remove(0)
                    
                    # Sort remaining solves
                    for i in range(len(tmpSolves)):
                        if tmpSolves[i] < 0:
                            tmpSolves[i] = 999999
                    tmpSolves = sorted(tmpSolves)
    
                    # Check Ao5
                    if (len(tmpSolves)) == 5:
                        if tmpSolves[3] == 999999:
                            if average != -1:
                                print('WARNING: DNF average for {} in {} {}'.format(name, eventName, self.date))
                        else:
                            diff = round(average - (tmpSolves[1] + tmpSolves[2] + tmpSolves[3]) / 3, 2)
                            if diff < -0.01 or diff > 0.01:
                                print('WARNING: Incorrect Ao5 for {} in {} {}'.format(name, eventName, self.date), average, diff, tmpSolves)
                    
                    # Check Mo3
                    if (len(tmpSolves)) == 3:
                        if tmpSolves[2] == 999999:
                            if average != -1:
                                print('WARNING: DNF mean for {} in {} {}'.format(name, eventName, self.date))
                        else:
                            diff = round(average - (tmpSolves[0] + tmpSolves[1] + tmpSolves[2]) / 3, 2)
                            if diff < -0.01 or diff > 0.01:
                                print('WARNING: Incorrect Mo3 for {} in {} {}'.format(name, eventName, self.date), average, diff, tmpSolves)
                
                    prevResult = thisResult
                    prevSingle = single

                rowNo += 1

## Main Code

Process all competitions

In [13]:
import glob
import time

pc1 = time.perf_counter()

In [14]:
# Process spreadsheets

datePattern = '[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]'
xlsxPattern = '*.xlsx'

persons = {}
competitions = {}

for xlsx in glob.glob(os.path.join(projdir, 'data', datePattern, xlsxPattern)):
    competition = Competition(xlsx)
    competition.processSheets()
    competition.saveEvents()
    competitions[competition.date] = competition
    
# Save profile pages
for person in persons:
    persons[person].saveProfile()

In [15]:
# Save competitors
for eventName in eventNames:
    docsDir = os.path.join(projdir, 'docs', getSafeName(eventName))
    if not os.path.exists(docsDir):
        os.makedirs(docsDir)

    outFile = os.path.join(docsDir, 'persons.md')

    event = Event(eventName)
        
    with open(outFile, 'w') as f:

        f.write('## {}\n'.format(title))
        f.write('### {} Competitors\n\n'.format(event.longName))

        for age in range(100, 0, -10):
            found = False

            for personName in sorted(persons):
                person = persons[personName]
                if person.age == age:
                    if eventName in person.bestSingles:
                        if found == False:
                            f.write('#### {}\n\n'.format(formatAgeLong(age)))
                            
                            f.write('| Name | Single |')
                            if eventName.lower() != 'fmc':
                                f.write(' Average |')
                            f.write('\n')
                            
                            f.write('| -- |')
                            if eventName.lower() != 'fmc':
                                f.write(' --: | --: |')
                            else:
                                f.write(' :--: |')
                            f.write('\n')
                            
                            found = True

                        link = '[{}](../persons/{}.md)'.format(person.name, getSafeName(person.name))
                        f.write('| {} |'.format(link))
                        f.write(' {} |'.format(formatResult(person.bestSingles[eventName].single, eventName)))
                        if eventName.lower() != 'fmc':
                            if eventName in person.bestAverages:
                                f.write(' {} |'.format(formatResult(person.bestAverages[eventName].average, eventName)))
                            else:
                                f.write(' DNF |')
                        f.write('\n')

            if found:
                f.write('\n')

        writeGoogleSiteTag(f)

In [16]:
# Save best singles
for eventName in eventNames:
    docsDir = os.path.join(projdir, 'docs', getSafeName(eventName))
    if not os.path.exists(docsDir):
        os.makedirs(docsDir)

    outFile = os.path.join(docsDir, 'singles.md')

    event = Event(eventName)
        
    with open(outFile, 'w') as f:

        f.write('## {}\n'.format(title))
        f.write('### {} Singles\n\n'.format(event.longName))

        for age in range(100, 0, -10):
            results = []
            for personName in sorted(persons):
                person = persons[personName]
                if person.age == age:
                    if eventName in person.bestSingles:
                        if person.bestSingles[eventName].single and person.bestSingles[eventName].single > 0:
                            results.append(person.bestSingles[eventName])
            
            if results:
                f.write('#### {}\n\n'.format(formatAgeLong(age)))
                f.write('| # | Name | Date | Single | Awards |')
                if eventName.lower() == 'fmc':
                    f.write(' Solution |\n')
                else:
                    f.write(' Video |\n')

                f.write('| :--: | -- | :--: |')
                if eventName.lower() != 'fmc':
                    f.write(' --: |')
                else:
                    f.write(' :--: |')
                f.write(' :--: | -- |\n')

                rank = 1
                prevSingle = 0
                for result in sorted(results, key=lambda result: result.single):
                    if result.single > prevSingle:
                        f.write('| {} |'.format(rank))
                    else:
                        f.write('| |')
                    link = '[{}](../persons/{}.md)'.format(result.name, getSafeName(result.name))
                    f.write(' {} |'.format(link))
                    f.write(' [{}]({}.md) |'.format(result.date, result.date))
                    f.write(' {} | {} |'.format(formatResult(result.single, eventName), result.awards))

                    if 'http' in result.link:
                        f.write(' [Link]({}) |\n'.format(result.link))
                    else:
                        f.write(' |\n')

                    prevSingle = result.single
                    rank += 1
                    
                f.write('\n')

        writeGoogleSiteTag(f)

In [17]:
# Save best averages
for eventName in eventNames:
    docsDir = os.path.join(projdir, 'docs', getSafeName(eventName))
    if not os.path.exists(docsDir):
        os.makedirs(docsDir)

    outFile = os.path.join(docsDir, 'averages.md')

    event = Event(eventName)
        
    with open(outFile, 'w') as f:

        f.write('## {}\n'.format(title))
        f.write('### {} Averages\n\n'.format(event.longName))

        for age in range(100, 0, -10):
            results = []
            for personName in sorted(persons):
                person = persons[personName]
                if person.age == age:
                    if eventName in person.bestAverages:
                        if person.bestAverages[eventName].average and person.bestAverages[eventName].average > 0:
                            results.append(person.bestAverages[eventName])
            
            if results:
                f.write('#### {}\n\n'.format(formatAgeLong(age)))
                f.write('| # | Name | Date | Average | Awards |')
                if eventName.lower() == 'fmc':
                    f.write(' Solution |\n')
                else:
                    f.write(' Video |\n')

                f.write('| :--: | -- | :--: |')
                if eventName.lower() != 'fmc':
                    f.write(' --: |')
                else:
                    f.write(' :--: |')
                f.write(' :--: | -- |\n')

                rank = 1
                prevAverage = 0
                for result in sorted(results, key=lambda result: result.average):
                    if result.average > prevAverage:
                        f.write('| {} |'.format(rank))
                    else:
                        f.write('| |')
                    link = '[{}](../persons/{}.md)'.format(result.name, getSafeName(result.name))
                    f.write(' {} |'.format(link))
                    f.write(' [{}]({}.md) |'.format(result.date, result.date))
                    f.write(' {} | {} |'.format(formatResult(result.average, eventName), result.awards))

                    if 'http' in result.link:
                        f.write(' [Link]({}) |\n'.format(result.link))
                    else:
                        f.write(' |\n')

                    prevAverage = result.average
                    rank += 1
                    
                f.write('\n')

        writeGoogleSiteTag(f)

In [18]:
# Save senior podiums
for eventName in eventNames:
    docsDir = os.path.join(projdir, 'docs', getSafeName(eventName))
    if not os.path.exists(docsDir):
        os.makedirs(docsDir)

    outFile = os.path.join(docsDir, 'README.md')

    event = Event(eventName)
        
    with open(outFile, 'w') as f:

        f.write('## {}\n'.format(title))
        f.write('### {} Senior Podiums\n'.format(event.longName))

        for competition in sorted(competitions, reverse = True):
            competition = competitions[competition]
            
            if eventName in competition.events:
                f.write('#### [{}]({}.md)\n\n'.format(competition.date, competition.date))
                results = competition.events[eventName].results
                
                f.write('| # | Name | Age | Single |')
                if eventName.lower() == 'fmc':
                    f.write(' Solution |\n')
                else:
                    f.write(' Average | Awards | Video |\n')

                f.write('| :--: | -- | :--: |')
                if eventName.lower() != 'fmc':
                    f.write(' --: | --: |')
                else:
                    f.write(' :--: |')
                f.write(' :--: | -- |\n')

                for result in results:
                    pos = 0
                    if '🥇' in result.awards:
                        pos = 1
                    if '🥈' in result.awards:
                        pos = 2
                    if '🥉' in result.awards:
                        pos = 3
                    if pos > 0:
                        link = '[{}](../persons/{}.md)'.format(result.name, getSafeName(result.name))
                        f.write('| {} | {} | {} |'.format(pos, link, formatAge(result.age)))
                        f.write(' {} |'.format(formatResult(result.single, eventName)))
                        if eventName.lower() != 'fmc':
                            f.write(' {} | {} |'.format(formatResult(result.average, eventName), result.awards))

                        if 'http' in result.link:
                            f.write(' [Link]({}) |\n'.format(result.link))
                        else:
                            f.write(' |\n')
                        
            f.write('\n')

        writeGoogleSiteTag(f)

In [19]:
# Save index page
outFile = os.path.join(projdir, 'docs', 'README.md')
with open(outFile, 'w') as f:
    f.write('## {}\n'.format(title))
    f.write('### Historical Archive\n')

    f.write('#### Individuals\n')

    eventList = []
    for eventName in eventNames:
        event = Event(eventName)
        eventList.append('[{}]({}/persons.md)'.format(event.shortName, getSafeName(eventName)))
    f.write('Competitors - {}\n\n'.format(', '.join(eventList)))
    
    eventList = []
    for eventName in eventNames:
        event = Event(eventName)
        eventList.append('[{}]({}/singles.md)'.format(event.shortName, getSafeName(eventName)))
    f.write('Best Singles - {}\n\n'.format(', '.join(eventList)))
    
    eventList = []
    for eventName in eventNames:
        event = Event(eventName)
        if eventName.lower() != 'fmc':
            eventList.append('[{}]({}/averages.md)'.format(event.shortName, getSafeName(eventName)))
    f.write('Best Averages - {}\n\n'.format(', '.join(eventList)))
    
    f.write('#### Podiums\n')

    eventList = []
    for eventName in eventNames:
        event = Event(eventName)
        eventList.append('[{}]({}/README.md)'.format(event.shortName, getSafeName(eventName)))
    f.write('Seniors - {}\n\n'.format(', '.join(eventList)))

    f.write('#### Results\n')

    for competition in sorted(competitions, reverse = True):
        competition = competitions[competition]
        eventList = []
        for eventName in eventNames:
            if eventName in competition.events:
                event = Event(eventName)
                eventList.append('[{}]({}/{}.md)'.format(event.shortName, getSafeName(eventName), competition.date))
        f.write('{} - {}\n\n'.format(competition.date, ', '.join(eventList)))
    
    f.write('#### Project Code\n')

    url = 'https://github.com/Logiqx/scw-comp'
    f.write('Python code for these reports can be found at [{}]({})\n'.format(url, url))

    writeGoogleSiteTag(f)

In [20]:
pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 2.53 seconds


## All Done!